# CallAPI

In [2]:
import time
from googleapiclient.discovery import build
import isodate
import json
import os

# 환경 변수에서 API 키 가져오기
API_KEY = os.getenv("YOUTUBE_API_KEY")
if not API_KEY:
    raise ValueError("API 키가 설정되지 않았습니다. 환경 변수 'YOUTUBE_API_KEY'를 설정하세요.")

# YouTube API 설정
youtube = build("youtube", "v3", developerKey=API_KEY)

# 카테고리 ID 설정
CATEGORIES = {
    "News & Politics": "25"
}

# 동영상 데이터 가져오기
def fetch_trending_videos(category_id, region_code="KR", max_results=200):
    videos = []
    next_page_token = None

    while len(videos) < max_results:
        try:
            request = youtube.videos().list(
                part="snippet,statistics,contentDetails",
                chart="mostPopular",
                regionCode=region_code,
                videoCategoryId=category_id,
                maxResults=min(50, max_results - len(videos)),
                pageToken=next_page_token
            )
            response = request.execute()

            for item in response.get("items", []):
                duration = isodate.parse_duration(item["contentDetails"]["duration"])
                duration_in_seconds = duration.total_seconds()  #초로 바꾸기기

                if duration_in_seconds > 80:  # 80초 이상의 동영상만 가져오기
                    videos.append({
                        "video_id": item["id"],
                        "title": item["snippet"]["title"],
                        "description": item["snippet"]["description"],
                        "tags": item["snippet"].get("tags", []),
                        "duration": str(duration),
                        "view_count": int(item["statistics"].get("viewCount", 0)),
                        "like_count": int(item["statistics"].get("likeCount", 0)),
                        "comment_count": int(item["statistics"].get("commentCount", 0)),
                        "category_id": category_id,
                    })

            next_page_token = response.get("nextPageToken")
            if not next_page_token:
                break

        except Exception as e:
            print(f"Error fetching videos: {e}")
            time.sleep(5)  # 잠시 대기 후 다시 시도

    return videos

# JSON 파일로 저장하는 함수
def save_to_json(data, filename):
    # data 폴더가 존재하지 않으면 생성
    if not os.path.exists('data'):
        os.makedirs('data')
    
    # 파일 경로를 data 폴더 아래로 설정
    filepath = os.path.join('data', filename)
    
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

# 실행
all_videos = {}

for category_name, category_id in CATEGORIES.items():
    print(f"Fetching trending videos for category: {category_name}")
    videos = fetch_trending_videos(category_id, region_code="KR", max_results=200)
    all_videos[category_name] = videos
    print(f"비디오 {len(videos)} 개 카테고리: {category_name} fetch 완료.")

# 결과를 하나의 JSON 파일로 저장
output_file = "raw_video_data.json"
save_to_json(all_videos, output_file)
print(f"데이터 저장 : data/{output_file}'")

# 결과 출력 예시
for category, videos in all_videos.items():
    print(f"\nCategory: {category}")
    for video in videos[:5]:
        print(f" - {video['title']} ({video['video_id']}), 조회수: {video['view_count']} 회, 좋아요: {video['like_count']} 개")

Fetching trending videos for category: News & Politics
비디오 60 개 카테고리: News & Politics fetch 완료.
데이터 저장 : data/raw_video_data.json'

Category: News & Politics
 - 김건희와 검찰을 끝까지 의심해야 하는 이유? (곽종근의 분노)｜풀버전 (l_MGS00btCo), 조회수: 985204 회, 좋아요: 57480 개
 - 미친 듯 날뛰던 이재명. 전한길 등장에 꼭꼭 숨었다 [주말 몰아보기] (xJbj4MRPXOs), 조회수: 1034657 회, 좋아요: 90036 개
 - 이상민 입 열자 尹 '휘청' "10시 KBS" 발언의 비밀 [뉴스.zip/MBC뉴스] (EMKYRHDCDC8), 조회수: 1188464 회, 좋아요: 22741 개
 - 尹 사건 맡은 중앙지법! 대형사건 터졌다![배승희 뉴스배송] (xYld5mZwuQU), 조회수: 846914 회, 좋아요: 141696 개
 - “극우가 점입가경”…민주당, 전한길 공개 비판 / 채널A / 김진의 돌직구쇼 (6sk_f6hx0OY), 조회수: 440676 회, 좋아요: 20699 개


In [3]:
def load_json(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        return json.load(f)

# JSON 파일 경로
input_file_path = "data/raw_video_data.json"

# JSON 데이터 로드
data = load_json(input_file_path)

def merge_text_fields(video):
    title = video.get("title", "")
    description = video.get("description", "")
    tags = " ".join(video.get("tags", []))  # 태그 리스트를 공백으로 연결

    # 병합
    merged_text = " ".join([title, description, tags])
    return merged_text.strip()  # 공백 제거

def extract_texts(data):
    texts = []
    for category, videos in data.items():
        for video in videos:
            merged_text = merge_text_fields(video)
            texts.append(merged_text)
    return texts

# 텍스트 데이터 추출
texts = extract_texts(data)

print(texts[:5])

["김건희와 검찰을 끝까지 의심해야 하는 이유? (곽종근의 분노)｜풀버전 #최욱 #매불쇼 #팟빵\n\n0:00 - 대기 시간\n17:19 - 방송 시작\n20:44 - 사이다 헤드라인 : 곽수산 X 장윤선 X 신인규 X 박지원 X 임미애 X 서영교\n1:49:59 - 세계로 뻗을레오 : 김준형 X 최광철\n2:30:42 - 이건 왜 이럴 과학 : 엑소 X 박태웅\n- 광고요정 : 김유정\n---------------------------------------------------------\n● 문제성 손발톱엔 케라셀!\n배송메시지에 '매불쇼'입력하면 케라셀풋크림 증정(2/28마감)\n➜ https://m.site.naver.com/1wO6u\n \n● 식후혈당 걱정엔 닥터엠당엔\n오늘 밤 12시까지 배송 메모에 '매불쇼 이벤트' 남겨주시면 2박스 구매시 1박스 더! (2+1)\n➜ https://smartstore.naver.com/mydrm/products/8236044676\n\n● 100% 순수 식물성 멜라토닌  벨나잇 으로 편안한 숙면~!\n2+1 진행 중,  매불쇼 기재하면 휴대용 7일분 추가 증정~!\n“문의전화 1533-1068”\n➜ https://m.site.naver.com/1B6jv\n---------------------------------------------------------\n광고 문의 : ad@podbbang.com\n출연, 사연 등 기타 문의 : mbshow@podbbang.com\n또는 인스타그램 @maebulshow 메시지 주세요 :)", "미친 듯 날뛰던 이재명. 전한길 등장에 꼭꼭 숨었다 [주말 몰아보기] 영상 ‘좋아요’와 ‘구독’은 큰 힘이 됩니다\n\n*‘굿모닝 대한민국’ 주말 몰아보기 영상 목록\n\n00:00:00 (월, 백지원)조기대선 노리는 모리배들, 전한길 선생 좀 보고 배워라\n00:26:06 (월, 강대규) 역사강사 전한길의 호소가 한국 국민을 전율케 한다\n00:37:27 (화, 김기흥) 내란 프레

In [4]:
def save_texts_to_file(texts, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for text in texts:
            f.write(text + '\n')

# 텍스트 파일로 저장
output_file_path = "data/merged_texts.txt"
save_texts_to_file(texts, output_file_path)
print(f"병합된 텍스트가 '{output_file_path}'에 저장되었습니다.")

병합된 텍스트가 'data/merged_texts.txt'에 저장되었습니다.


In [5]:
def load_text_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return f.readlines()
    
# 텍스트 파일 경로
input_file_path = "data/merged_texts.txt"

texts = load_text_file(input_file_path)

print(texts[:5])



['김건희와 검찰을 끝까지 의심해야 하는 이유? (곽종근의 분노)｜풀버전 #최욱 #매불쇼 #팟빵\n', '\n', '0:00 - 대기 시간\n', '17:19 - 방송 시작\n', '20:44 - 사이다 헤드라인 : 곽수산 X 장윤선 X 신인규 X 박지원 X 임미애 X 서영교\n']


In [10]:
import re
import emoji


# 텍스트 전처리 함수
def preprocess_text(text):
    if not isinstance(text, str):
        text = str(text)  # 문자열로 변환

    # 1. HTML 태그 제거
    text = re.sub(r"<[^>]+>", "", text)

    # 2. URL 제거
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)

    # 3. 이메일 제거
    text = re.sub(r"\S+@\S+\.\S+", "", text)

    # 4. 숫자 제거 (명확히 숫자만 제거)
    text = re.sub(r"\d+", "", text)

    # 5. 반복된 ㅋ, ㅎ, ㅠ, ㅜ 등 제거
    text = re.sub(r"[ㅋㅎㅠㅜ]+", "", text)

    # 6. 반복된 점(...) 제거
    text = re.sub(r"\.\.+", ".", text)
    
    # 7. 불용어 제거
    stopwords = set(["시", "분,"])
    text = " ".join([word for word in text.split() if word not in stopwords])

    # 8. 이모지 제거
    text = emoji.replace_emoji(text, replace="")
    
    # 연속된 공백을 하나의 공백으로 변환
    text = re.sub(r"\s+", " ", text)

    # 9. 특수문자 및 영어 알파벳 제거
    text = re.sub(r"[^\w\s가-힣]", "", text)  # 영어 알파벳 포함 특수문자 제거
    text = re.sub(r"[a-zA-Z]", "", text)     # 영어 알파벳 제거


    return text

# 텍스트 데이터 전처리
preprocessed_texts = [preprocess_text(line) for line in texts]

print(preprocessed_texts[:5])

save_texts_to_file(preprocessed_texts, "data/preprocessed_texts.txt")




['김건희와 검찰을 끝까지 의심해야 하는 이유 곽종근의 분노풀버전 최욱 매불쇼 팟빵', '', '  대기 시간', '  방송 시작', '  사이다 헤드라인  곽수산  장윤선  신인규  박지원  임미애  서영교']
